<a href="https://colab.research.google.com/github/CYRCHER/-bothub-io-.github.io./blob/main/malaria_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

# Install required libraries
!pip install tensorflow==2.6.0
!pip install keras==2.6.0
!pip install opencv-python==4.5.3.56


Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.4/458.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 61.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 90.8 MB/s eta 0:00:00
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30692 sha256=a7fc3da19e389bc7bd3151ec56d89f9774a733a54c5ea02996749563855cf62c
  Stored in directory: /root/.cache/pip/wheels/3a/ce/7a/27094f689461801c934296d07078773603663dfcaca63bb064
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4845 sha256=5b17126

In [ ]:
import os
import cv2
import numpy as np
import zipfile
import io

# Define the paths to the dataset zip file and the directories to save the images and masks
# zip_file_path = '/content/drive/My Drive/MalariaDataset.zip'
image_dir = '/content/drive/My Drive/MalariaDataset/Images'
mask_dir = '/content/drive/My Drive/MalariaDataset/Masks'

# Extract the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
#     zip_file.extractall('/content/')

# Create the directories for the images and masks
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
if not os.path.exists(mask_dir):
    os.makedirs(mask_dir)

In [ ]:
import random
import shutil

# Define the directories for the train, validation, and test sets
train_dir = '/content/train/'
val_dir = '/content/val/'
test_dir = '/content/test/'

# Create the directories for the train, validation, and test sets
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

In [ ]:
# Define the fraction of the dataset to use for training, validation, and test sets
train_frac = 0.7
val_frac = 0.2
test_frac = 0.1

# Get the list of image filenames
image_filenames = os.listdir(image_dir)
# Shuffle the list
random.shuffle(image_filenames)

# Split the dataset into train, validation, and test sets
train_size = int(len(image_filenames) * train_frac)
val_size = int(len(image_filenames) * val_frac)
test_size = int(len(image_filenames) * test_frac)

train_images = image_filenames[:train_size]
val_images = image_filenames[train_size:train_size+val_size]
test_images = image_filenames[train_size+val_size:]

In [ ]:
# Copy the images and masks to the appropriate directories
for image_name in train_images:
    mask_name = image_name.split('.')[0] +'_mask' + '.png'
    shutil.copy(image_dir +'/'+ image_name, train_dir + image_name)
    shutil.copy(mask_dir + '/'+  mask_name, train_dir + mask_name)
    
for image_name in val_images:
  mask_name = image_name.split('.')[0] +'_mask' + '.png'
  shutil.copy(image_dir +'/'+ image_name, val_dir + image_name)
  shutil.copy(mask_dir +'/'+ mask_name, val_dir + mask_name)

for image_name in test_images:
  mask_name = image_name.split('.')[0] +'_mask' + '.png'
  shutil.copy(image_dir +'/'+ image_name, test_dir + image_name)
  shutil.copy(mask_dir + '/'+ mask_name, test_dir + mask_name)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose, multiply, add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from google.colab import drive

In [ ]:
# Build the UNet Model
# Define the encoder block
def encoder_block(inputs, filters, pool=True):
    conv = Conv2D(filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv = Conv2D(filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv)
    if pool:
        pool = MaxPooling2D(pool_size=(2, 2))(conv)
        return conv, pool
    else:
        return conv

# Define the decoder block
def decoder_block(inputs, skip_features, filters):
    up = Conv2DTranspose(filters, 2, strides=(2, 2), padding='same')(inputs)
    attention = attention_block(skip_features, up)
    concat = concatenate([up, attention])
    conv = Conv2D(filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(concat)
    conv = Conv2D(filters, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv)
    return conv

# Define the attention block
def attention_block(x, g, inter_channel=2):
    theta_x = Conv2D(inter_channel, 1, strides=1, padding='same')(x)
    phi_g = Conv2D(inter_channel, 1, strides=1, padding='same')(g)
    f = tf.keras.activations.relu(add([theta_x, phi_g]))
    psi_f = Conv2D(1, 1, strides=1, padding='same')(f)
    sigmoid_f = tf.keras.activations.sigmoid(psi_f)
    attention = multiply([x, sigmoid_f])
    return attention
  
  # Define the UNet model with attention blocks
def unet_with_attention(input_size=(256, 256, 3)):
    # Encoder
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    # Bridge
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2DTranspose(512, kernel_size=2, strides=2, padding='same')(drop5)
    up6 = concatenate([up6, drop4])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)
    conv6 = attention_block(conv6, drop4)

    up7 = Conv2DTranspose(256, kernel_size=2, strides=2, padding='same')(conv6)
    up7 = concatenate([up7, conv3])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)
    conv7 = attention_block(conv7, conv3)

    up8 = Conv2DTranspose(128, kernel_size=2, strides=2, padding='same')(conv7)
    up8 = concatenate([up8, conv2])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)
    conv8 = attention_block(conv8, conv2)

    up9 = Conv2DTranspose(64, kernel_size=2, strides=2, padding='same')(conv8)
    up9 = concatenate([up9, conv1])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    att9 = attention_block(conv9, conv1)
    conv9 = concatenate([conv9, att9])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    
    # Output layer
    output = Conv2D(1, 1, activation='sigmoid', name='output')(conv9)
    
    # Create the model
    model = Model(inputs=[inputs], outputs=[output])

    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Print the model summary
    model.summary()

    return model

In [ ]:
# Train the UNet Model with Attention Mechanisms
model = unet_with_attention()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
# Preprocess the Data
# Define the image and mask size
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Define the batch size
BATCH_SIZE = 32

# Define the path to the dataset
data_dir = val_images
print(data_dir)

# Define the train and test sets using the ImageDataGenerator class
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        val_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='binary',
        shuffle=False)

[]
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from math import ceil

# define number of epochs
num_epochs = 10

# define batch size
batch_size = 32

# define steps per epoch for training data
steps_per_epoch_train = ceil(len(train_images) / batch_size)

# define validation data generator
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size,
    class_mode='categorical'
)

# define steps per epoch for validation data
steps_per_epoch_val = ceil(len(val_images) / batch_size)

# define checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_train,
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=steps_per_epoch_val,
    callbacks=[checkpoint_callback]
)

Found 0 images belonging to 0 classes.


ValueError: ignored

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

ValueError: ignored

In [ ]:
from matplotlib import pyplot as plt

# Get a batch of test images and masks
test_images, test_masks = next(test_generator)

# Predict the masks for the test images using the trained model
predicted_masks = model.predict(test_images)

# Display the test images, true masks, and predicted masks
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(15, 9))

for i in range(5):
    axes[0][i].imshow(test_images[i])
    axes[0][i].set_title('Test Image')
    axes[0][i].axis('off')
    
    axes[1][i].imshow(test_masks[i].squeeze(), cmap='gray')
    axes[1][i].set_title('True Mask')
    axes[1][i].axis('off')
    
    axes[2][i].imshow(predicted_masks[i].squeeze(), cmap='gray')
    axes[2][i].set_title('Predicted Mask')
    axes[2][i].axis('off')

plt.tight_layout()
plt.show()


ValueError: ignored